In [1]:
import numpy as np
import scanpy as sc
import anndata
import scvi
from scib_metrics.benchmark import Benchmarker
import torch
from functions import *
import scib_metrics
%matplotlib inline

/local/data/scvi-tools/scvi/_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
/local/data/scvi-tools/scvi/_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please pass in `pin_memory` to the data loaders instead.
  self.dl_pin_memory_gpu_training = (
/local/data/anaconda3/envs/BT/lib/python3.11/site-packages/numba/np/ufunc/dufunc.py:84: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  dispatcher = jit(_target='npyufunc',
/local/data/anaconda3/envs/BT/lib/python3.11/site-pac

In [2]:
adata = anndata.read_h5ad(filename="data/adataLung4.h5ad")
adata

AnnData object with n_obs × n_vars = 32472 × 4000
    obs: 'dataset', 'location', 'nGene', 'nUMI', 'patientGroup', 'percent.mito', 'protocol', 'sanger_type', 'size_factors', 'sampling_method', 'batch', 'cell_type', 'donor'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection'
    uns: 'hvg', 'pca'
    obsm: 'Scanorama', 'Unintegrated', 'X_pca'
    varm: 'PCs'
    layers: 'counts'

In [3]:
scvi.model.SCVI.setup_anndata(adata, layer="counts")

INFO:jax._src.xla_bridge:Unable to initialize backend 'cuda': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': INVALID_ARGUMENT: TpuPlatform is not available.


In [11]:
model = scvi.model.SCVI(adata)

In [14]:
model.train(plan_kwargs={"max_kl_weight":10,"n_epochs_kl_warmup":200})

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 1/246:   0%|          | 0/246 [00:00<?, ?it/s]

Epoch 2/246:   0%|          | 1/246 [00:01<06:22,  1.56s/it, v_num=1, train_loss_step=979, train_loss_epoch=947]

In [6]:
vaeSD = scvi.model.SCVI(adata, prior_distribution="sdnormal")
vaeMG = scvi.model.SCVI(adata, prior_distribution="mixofgaus")
vaeVP = scvi.model.SCVI(adata, prior_distribution="vamp")
vaeNF = scvi.model.SCVI(adata, prior_distribution="normalflow",prior_kwargs={"num_layers":8})

In [7]:
logger = TensorBoardLogger(save_dir="lightning_logs",name="sd_kl_max_exp_5_600")
vaeSD.train(plan_kwargs={"max_kl_weight":5,"n_epochs_kl_warmup":400},check_val_every_n_epoch=5,max_epochs=600,logger=logger)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: lightning_logs/sd_kl_max_exp_5_600
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 600/600: 100%|██████████| 600/600 [17:33<00:00,  1.88s/it, v_num=0, train_loss_step=888, train_loss_epoch=927]    

`Trainer.fit` stopped: `max_epochs=600` reached.


Epoch 600/600: 100%|██████████| 600/600 [17:33<00:00,  1.76s/it, v_num=0, train_loss_step=888, train_loss_epoch=927]


In [8]:
logger = TensorBoardLogger(save_dir="lightning_logs",name="mog_kl_max_exp_5_600")
vaeMG.train(plan_kwargs={"max_kl_weight":5,"n_epochs_kl_warmup":400},check_val_every_n_epoch=5,max_epochs=600,logger=logger)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Missing logger folder: lightning_logs/mog_kl_max_exp_5_600
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 600/600: 100%|██████████| 600/600 [20:31<00:00,  2.16s/it, v_num=0, train_loss_step=911, train_loss_epoch=915]    

`Trainer.fit` stopped: `max_epochs=600` reached.


Epoch 600/600: 100%|██████████| 600/600 [20:31<00:00,  2.05s/it, v_num=0, train_loss_step=911, train_loss_epoch=915]


In [9]:
logger = TensorBoardLogger(save_dir="lightning_logs",name="vamp_kl_max_exp_5_600")
vaeVP.train(plan_kwargs={"max_kl_weight":5,"n_epochs_kl_warmup":400},check_val_every_n_epoch=5,max_epochs=600,logger=logger)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: lightning_logs/vamp_kl_max_exp_5_600
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 600/600: 100%|██████████| 600/600 [23:02<00:00,  2.36s/it, v_num=0, train_loss_step=901, train_loss_epoch=915]    

`Trainer.fit` stopped: `max_epochs=600` reached.


Epoch 600/600: 100%|██████████| 600/600 [23:02<00:00,  2.30s/it, v_num=0, train_loss_step=901, train_loss_epoch=915]


In [10]:
logger = TensorBoardLogger(save_dir="lightning_logs",name="flow_kl_max_exp_5_600")
vaeNF.train(plan_kwargs={"max_kl_weight":5,"n_epochs_kl_warmup":400},check_val_every_n_epoch=5,max_epochs=600,logger=logger)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: lightning_logs/flow_kl_max_exp_5_600


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 600/600: 100%|██████████| 600/600 [37:58<00:00,  3.82s/it, v_num=0, train_loss_step=965, train_loss_epoch=915]    

`Trainer.fit` stopped: `max_epochs=600` reached.


Epoch 600/600: 100%|██████████| 600/600 [37:58<00:00,  3.80s/it, v_num=0, train_loss_step=965, train_loss_epoch=915]


In [11]:
adataSD = adata.copy()
adataSD.obsm["scVI"] = vaeSD.get_latent_representation()
adataMG = adata.copy()
adataMG.obsm["scVI"] = vaeMG.get_latent_representation()
adataVP = adata.copy()
adataVP.obsm["scVI"] = vaeVP.get_latent_representation()
adataNF = adata.copy()
adataNF.obsm["scVI"] = vaeNF.get_latent_representation()

In [27]:
def getBenmarkRes(vae):
    adatac = adata.copy()
    adatac.obsm["scVI"] = vae.get_latent_representation()
    bm = Benchmarker(
            adatac,
            batch_key="batch",
            label_key="cell_type",
            embedding_obsm_keys=["scVI"],
            bio_conservation_metrics=scib_metrics.benchmark.BioConservation(isolated_labels=False, nmi_ari_cluster_labels_leiden=True, nmi_ari_cluster_labels_kmeans=True, silhouette_label=False, clisi_knn=False),
            batch_correction_metrics=scib_metrics.benchmark.BatchCorrection(silhouette_batch=False, ilisi_knn=False, kbet_per_label=True, graph_connectivity=False, pcr_comparison=False),
            n_jobs=16
    )
    bm.benchmark()
    return bm.get_results(min_max_scale=False)

In [28]:
getBenmarkRes(vaeSD)

Embeddings: 100%|██████████| 1/1 [00:20<00:00, 20.23s/it]


,Leiden NMI,Leiden ARI,KMeans NMI,KMeans ARI,KBET,Batch correction,Bio conservation,Total
Embedding,,,,,,,,
scVI,0.73135,0.410593,0.700247,0.502,0.216262,0.216262,0.586047,0.438133
Metric Type,Bio conservation,Bio conservation,Bio conservation,Bio conservation,Batch correction,Aggregate score,Aggregate score,Aggregate score


In [29]:
getBenmarkRes(vaeMG)

Embeddings: 100%|██████████| 1/1 [00:19<00:00, 19.96s/it]


,Leiden NMI,Leiden ARI,KMeans NMI,KMeans ARI,KBET,Batch correction,Bio conservation,Total
Embedding,,,,,,,,
scVI,0.735749,0.422247,0.682725,0.433631,0.252565,0.252565,0.568588,0.442179
Metric Type,Bio conservation,Bio conservation,Bio conservation,Bio conservation,Batch correction,Aggregate score,Aggregate score,Aggregate score


In [30]:
getBenmarkRes(vaeVP)

Embeddings: 100%|██████████| 1/1 [00:20<00:00, 20.27s/it]


,Leiden NMI,Leiden ARI,KMeans NMI,KMeans ARI,KBET,Batch correction,Bio conservation,Total
Embedding,,,,,,,,
scVI,0.731624,0.421702,0.655362,0.418492,0.225359,0.225359,0.556795,0.42422
Metric Type,Bio conservation,Bio conservation,Bio conservation,Bio conservation,Batch correction,Aggregate score,Aggregate score,Aggregate score


In [31]:
getBenmarkRes(vaeNF)

Embeddings: 100%|██████████| 1/1 [00:19<00:00, 19.63s/it]


,Leiden NMI,Leiden ARI,KMeans NMI,KMeans ARI,KBET,Batch correction,Bio conservation,Total
Embedding,,,,,,,,
scVI,0.728135,0.405091,0.677736,0.452499,0.224709,0.224709,0.565865,0.429403
Metric Type,Bio conservation,Bio conservation,Bio conservation,Bio conservation,Batch correction,Aggregate score,Aggregate score,Aggregate score
